In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from spx_history import *

- date range

In [3]:
start_dt = dt.date(2021,1,1)
end_dt = dt.date.today()

In [6]:
tu = pd.read_csv('data/spy_cst.csv')

In [14]:
tickers = tu[tu['Ticker'] != '-']['Ticker'].unique().tolist()

- split data

In [25]:
def download_data(fun_to_d, tickers, start_dt, end_dt):
	fun_to_name = {
		get_splits : "split",
		get_dividends : "div",
		get_raw_intraday : "intraday"
	}

	# dataset name
	data_name = fun_to_name[fun_to_d]

	# create directory if doesnt exist
	csv_path_dir = f"data/{data_name}/"
	Path(csv_path_dir).mkdir(parents=True, exist_ok=True)

	# failed tickers
	failed_tickers = {}

	for sym in tickers:
		csv_file_path = f"{csv_path_dir}/{sym}.csv"

		if Path(csv_file_path).is_file():
			qt.log.info(f"csv file for {sym} already exists. skip")
			continue
	
		else:
			try:
				qt.log.info(f"querying {data_name} data for symbol {sym}")
				t = fun_to_d(sym, start_dt, end_dt)
				if len(t) != 0:
					qt.log.info(f"saving {t.shape} rows for symbol {sym} to file {csv_file_path}")
					t.to_csv(csv_file_path, index=False)
				else:
					assert False, "0 rows returned"
			
			except Exception as e:
				qt.log.warning(f"error occurred with ticker : {sym}, error : {e}")
				failed_tickers[sym] = e
	
	return failed_tickers

In [19]:
failed_tickers = download_data(get_dividends, tickers, start_dt, end_dt)

[JUSTY.LOG]	2026-02-05 22:08:53,177 - qt.common.help - INFO - csv file for NVDA already exists. skip
[JUSTY.LOG]	2026-02-05 22:08:53,180 - qt.common.help - INFO - csv file for AAPL already exists. skip
[JUSTY.LOG]	2026-02-05 22:08:53,181 - qt.common.help - INFO - csv file for MSFT already exists. skip


[JUSTY.LOG]	2026-02-05 22:08:53,182 - qt.common.help - INFO - csv file for AMZN already exists. skip
[JUSTY.LOG]	2026-02-05 22:08:53,183 - qt.common.help - INFO - csv file for GOOGL already exists. skip
[JUSTY.LOG]	2026-02-05 22:08:53,184 - qt.common.help - INFO - csv file for GOOG already exists. skip
[JUSTY.LOG]	2026-02-05 22:08:53,186 - qt.common.help - INFO - csv file for META already exists. skip
[JUSTY.LOG]	2026-02-05 22:08:53,188 - qt.common.help - INFO - csv file for AVGO already exists. skip
[JUSTY.LOG]	2026-02-05 22:08:53,188 - qt.common.help - INFO - csv file for TSLA already exists. skip
[JUSTY.LOG]	2026-02-05 22:08:53,189 - qt.common.help - INFO - csv file for BRK.B already exists. skip
[JUSTY.LOG]	2026-02-05 22:08:53,190 - qt.common.help - INFO - csv file for LLY already exists. skip
[JUSTY.LOG]	2026-02-05 22:08:53,191 - qt.common.help - INFO - csv file for JPM already exists. skip
[JUSTY.LOG]	2026-02-05 22:08:53,192 - qt.common.help - INFO - csv file for XOM already exis

- split data

In [23]:
failed_tickers_split = download_data(get_splits, tickers, start_dt, end_dt)

[JUSTY.LOG]	2026-02-05 23:07:40,968 - qt.common.help - INFO - csv file for NVDA already exists. skip
[JUSTY.LOG]	2026-02-05 23:07:40,969 - qt.common.help - INFO - csv file for AAPL already exists. skip
[JUSTY.LOG]	2026-02-05 23:07:40,970 - qt.common.help - INFO - csv file for MSFT already exists. skip
[JUSTY.LOG]	2026-02-05 23:07:40,970 - qt.common.help - INFO - csv file for AMZN already exists. skip
[JUSTY.LOG]	2026-02-05 23:07:40,970 - qt.common.help - INFO - csv file for GOOGL already exists. skip
[JUSTY.LOG]	2026-02-05 23:07:40,971 - qt.common.help - INFO - csv file for GOOG already exists. skip
[JUSTY.LOG]	2026-02-05 23:07:40,972 - qt.common.help - INFO - csv file for META already exists. skip
[JUSTY.LOG]	2026-02-05 23:07:40,972 - qt.common.help - INFO - csv file for AVGO already exists. skip
[JUSTY.LOG]	2026-02-05 23:07:40,973 - qt.common.help - INFO - csv file for TSLA already exists. skip
[JUSTY.LOG]	2026-02-05 23:07:40,973 - qt.common.help - INFO - csv file for BRK.B already e

In [24]:
failed_tickers_split

{}

- intraday data

In [27]:
get_raw_intraday("META", start_dt=start_dt, end_dt=end_dt)

[JUSTY.LOG]	2026-02-05 23:28:55,537 - qt.common.help - INFO - querying intraday data for symbol META from 2021-01-01 to 2021-04-11
[JUSTY.LOG]	2026-02-05 23:29:00,867 - qt.common.help - INFO - querying intraday data for symbol META from 2021-04-11 to 2021-07-20
[JUSTY.LOG]	2026-02-05 23:29:06,630 - qt.common.help - INFO - querying intraday data for symbol META from 2021-07-20 to 2021-10-28
[JUSTY.LOG]	2026-02-05 23:29:11,147 - qt.common.help - INFO - querying intraday data for symbol META from 2021-10-28 to 2022-02-05
[JUSTY.LOG]	2026-02-05 23:29:15,736 - qt.common.help - INFO - querying intraday data for symbol META from 2022-02-05 to 2022-05-16
[JUSTY.LOG]	2026-02-05 23:29:25,787 - qt.common.help - INFO - querying intraday data for symbol META from 2022-05-16 to 2022-08-24
[JUSTY.LOG]	2026-02-05 23:29:36,008 - qt.common.help - INFO - querying intraday data for symbol META from 2022-08-24 to 2022-12-02
[JUSTY.LOG]	2026-02-05 23:29:40,489 - qt.common.help - INFO - querying intraday dat

,timestamp,gmtoffset,open,high,low,close,volume
datetime,,,,,,,
2021-01-01 00:11:00+00:00,1609459860,0,272.80,272.80,272.80,272.80,267
2021-01-01 00:17:00+00:00,1609460220,0,272.66,272.66,272.66,272.66,420
2021-01-01 00:18:00+00:00,1609460280,0,272.78,272.78,272.78,272.78,510
2021-01-01 00:25:00+00:00,1609460700,0,272.75,272.75,272.75,272.75,110
2021-01-01 00:36:00+00:00,1609461360,0,272.65,272.65,272.65,272.65,360
...,...,...,...,...,...,...,...
2026-02-04 23:56:00+00:00,1770249360,0,671.35,671.50,670.65,670.67,593
2026-02-04 23:57:00+00:00,1770249420,0,671.00,671.48,670.70,671.25,980
2026-02-04 23:58:00+00:00,1770249480,0,671.01,671.50,670.70,670.90,830


In [28]:
failed_tickers_intraday = download_data(get_raw_intraday, tickers, start_dt, end_dt)

[JUSTY.LOG]	2026-02-06 01:04:57,319 - qt.common.help - INFO - querying intraday data for symbol NVDA
[JUSTY.LOG]	2026-02-06 01:04:57,359 - qt.common.help - INFO - querying intraday data for symbol NVDA from 2021-01-01 to 2021-04-11
[JUSTY.LOG]	2026-02-06 01:05:02,499 - qt.common.help - INFO - querying intraday data for symbol NVDA from 2021-04-11 to 2021-07-20
[JUSTY.LOG]	2026-02-06 01:05:08,204 - qt.common.help - INFO - querying intraday data for symbol NVDA from 2021-07-20 to 2021-10-28
[JUSTY.LOG]	2026-02-06 01:05:13,136 - qt.common.help - INFO - querying intraday data for symbol NVDA from 2021-10-28 to 2022-02-05
[JUSTY.LOG]	2026-02-06 01:05:22,974 - qt.common.help - INFO - querying intraday data for symbol NVDA from 2022-02-05 to 2022-05-16
[JUSTY.LOG]	2026-02-06 01:05:31,109 - qt.common.help - INFO - querying intraday data for symbol NVDA from 2022-05-16 to 2022-08-24
[JUSTY.LOG]	2026-02-06 01:05:36,315 - qt.common.help - INFO - querying intraday data for symbol NVDA from 2022-08

: 